In [1]:
import keras
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras.applications.mobilenet_v2 import decode_predictions
from keras.layers.normalization import BatchNormalization

from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow
from time import time

import pandas

import pickle as pk

from keras.wrappers.scikit_learn import KerasClassifier

import numpy as np

seed = 7

Using TensorFlow backend.


## Hyperparameter runing on mobilenet with RMSprop.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1846044515277481367
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11280557671
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7768095235545444265
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [3]:
dir(keras.applications.mobilenet_v2)

['MobileNetV2',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'absolute_import',
 'decode_predictions',
 'division',
 'keras_modules_injection',
 'mobilenet_v2',
 'preprocess_input',
 'print_function']

In [4]:
np.random.seed(seed)

In [5]:
data_dir = '/data/oxford102/train'
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
img_width, img_height = 256, 256
batch_size = 128
epochs = 100
nr_categories = n_categories = 102
nb_train_samples = 4604
nb_validation_samples = 1094 

In [6]:
def get_image_generator(input_processor, img_aug=False):
    if not img_aug:
        train_val_datagen = ImageDataGenerator(preprocessing_function=None,
                                               rescale=1./255, 
                                           validation_split=0.2)
    else: 
        train_val_datagen = ImageDataGenerator(
            preprocessing_function=None,
            rescale=1./255, 
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            validation_split=0.2)
        
    return train_val_datagen

In [7]:
ImageDataGenerator(
    featurewise_center=False, 
    samplewise_center=False, 
    featurewise_std_normalization=False, 
    samplewise_std_normalization=False, 
    zca_whitening=False, 
    zca_epsilon=1e-06, 
    rotation_range=0, 
    width_shift_range=0.0, 
    height_shift_range=0.0, 
    brightness_range=None,
    shear_range=0.0, 
    zoom_range=0.0,
    channel_shift_range=0.0, 
    fill_mode='nearest',
    cval=0.0, 
    horizontal_flip=False, 
    vertical_flip=False, 
    rescale=None, 
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0, 
    dtype=None)

In [8]:
def get_generators(batch_size, image_size, input_processor, img_aug=False):

    img_width, img_height = image_size
    
    train_val_datagen = get_image_generator(input_processor, img_aug)

    train_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            batch_size=batch_size,
            subset="training",
            class_mode='categorical')

    validation_generator = train_val_datagen.flow_from_directory(
            train_data_dir,  # this is the target directory
            target_size=(img_width, img_height),  # all images will be resized to 250x250
            subset="validation",
            batch_size=batch_size,
            class_mode='categorical')
    
    return train_generator, validation_generator


In [9]:
def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(n_categories, activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [10]:
get_benchmark_model(True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 64)        18496     
__________

(<keras.engine.sequential.Sequential at 0x7f9469882978>, None)

In [10]:
#from keras import regularizers

def get_model(network_name="inception_resnet_v2", image_size=(256, 256), verbose=False):
    k.set_learning_phase(0)

    img_width, img_height = image_size
    if network_name == "benchmark":
        return get_benchmark_model(img_size=image_size)
    if network_name == "vgg16":
        base_model = keras.applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.vgg16.preprocess_input
    elif network_name == "vgg19":
        base_model = keras.applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.vgg19.preprocess_input
    elif network_name == "inception_resnet_v2":
        base_model = keras.applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.inception_resnet_v2.preprocess_input
    elif network_name == "mobilenet_v2":
        base_model = keras.applications.mobilenet_v2.MobileNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
        input_processor = applications.mobilenet_v2.preprocess_input
    else:
        raise Exception("check your network name")

    for layer in base_model.layers[:]:
        layer.trainable = False

        #Adding custom Layers 
    k.set_learning_phase(1)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu", 
              #kernel_regularizer=regularizers.l2(0.01),
             #       activity_regularizer=regularizers.l1(0.001)
             )(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x, training=True)
    #x = Dense(102, activation="relu")(x)
    predictions = Dense(nr_categories, activation="softmax")(x)

    _model = Model(input = base_model.input, output = predictions)
    if verbose:
        _model.summary()
    return _model, input_processor

In [10]:
# Hyperparameter optimization

In [ ]:
optimizers

In [ ]:
optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

In [ ]:
optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

In [ ]:
optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [ ]:
optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)

In [ ]:
optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

In [10]:
#import talos as ta

# p = {'lr': (0.001, 0.01, 0.1),
#      'batch_size': (128),
#      'epochs': [150],
#      'dropout': (0, 0.5, 5),
#      'weight_regulizer':[None],
#      'emb_output_dims': [None],
#      'shape':['brick','long_funnel'],
#      'optimizer': [Adam, Nadam, RMSprop],
#      'losses': [logcosh, binary_crossentropy],
#      'activation':[relu, elu],
#      'last_activation': [sigmoid]}

In [11]:
def train_model(params, _model, generators):
    model_name = params["network_name"]
    num_train_img = 4604
    num_val_img = 1094

    np.random.seed(seed)
    log_time = time()
    params['log_time'] = log_time
    batch_size = params.get("batch_size")

    train_generator, validation_generator = generators
    _model.compile(loss = "categorical_crossentropy", optimizer = keras.optimizers.SGD(), metrics=["accuracy"])


    base = '/data/oxford102/experiments'
    path = os.path.join(base, str(log_time))
    checkpoint = ModelCheckpoint(os.path.join(path, "{}_{}.h5".format(model_name, log_time)), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
    tensorboard = TensorBoard(log_dir="logs/{}".format(log_time), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = CSVLogger(os.path.join(path, "{}_{}.csv".format(model_name, log_time)), append=True, separator=';')

    try:
        if not os.path.exists(path):
            os.makedirs(path)
        history_callback = _model.fit_generator(
                train_generator,
                steps_per_epoch=num_train_img // params["batch_size"] ,
                epochs=params["epochs"],
                validation_data=validation_generator,
                validation_steps=num_val_img // params["batch_size"],
                callbacks = [checkpoint, early, tensorboard, csv_logger])
    except Exception as e:
        raise(e)
    finally:
        params.pop("optimizer")
        pk.dump(params, open("experimental_params/experiments_{}.pk".format(log_time), "wb"), protocol=pk.HIGHEST_PROTOCOL)
        _model.save_weights(os.path.join(path, 'model_{}_weights_final_{}.h5'.format(model_name, log_time)))  # always save your weights after training or during training
        print(params)
        params

In [12]:
params = {'network_name': "benchmark",
         'image_aug': False,
         'optimizer_name': "RMSProp", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [28]:
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 19s 1s/step - loss: 15.9329 - acc: 0.0110 - val_loss: 15.9450 - val_acc: 0.0107

Epoch 00001: val_acc improved from -inf to 0.01074, saving model to /data/oxford102/experiments/1548037535.6415188/benchmark_1548037535.6415188.h5
Epoch 2/100
17/17 [==============================] - 17s 979ms/step - loss: 15.9551 - acc: 0.0101 - val_loss: 15.9012 - val_acc: 0.0135

Epoch 00002: val_acc improved from 0.01074 to 0.01346, saving model to /data/oxford102/experiments/1548037535.6415188/benchmark_1548037535.6415188.h5
Epoch 3/100
17/17 [==============================] - 15s 867ms/step - loss: 15.9845 - acc: 0.0083 - val_loss: 16.0180 - val_acc: 0.0062

Epoch 00003: val_acc did not improve from 0.01346
Epoch 4/100
17/17 [==============================] - 14s 847ms/step - loss: 15.9033 - acc: 0.0133 - val_loss: 15.9679 - val_acc: 0.0093

Epoch 00004: val_acc

In [29]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "RMSProp", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 128,
         'epochs': 100,
         'image_size': (224, 224),
         'log_time': None}

In [30]:
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_optimizer = optimizers.RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay)
params['optimizer'] = _optimizer
_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
17/17 [==============================] - 14s 816ms/step - loss: 15.6817 - acc: 0.0253 - val_loss: 15.6459 - val_acc: 0.0293

Epoch 00001: val_acc improved from -inf to 0.02930, saving model to /data/oxford102/experiments/1548037786.9805357/benchmark_1548037786.9805357.h5
Epoch 2/100
17/17 [==============================] - 15s 864ms/step - loss: 15.6070 - acc: 0.0317 - val_loss: 15.6342 - val_acc: 0.0300

Epoch 00002: val_acc improved from 0.02930 to 0.03002, saving model to /data/oxford102/experiments/1548037786.9805357/benchmark_1548037786.9805357.h5
Epoch 3/100
17/17 [==============================] - 13s 761ms/step - loss: 15.6875 - acc: 0.0267 - val_loss: 15.7010 - val_acc: 0.0259

Epoch 00003: val_acc did not improve from 0.03002
Epoch 4/100
17/17 [==============================] - 13s 773ms/step - loss: 15.6588 - acc: 0.0285 - val_loss: 15.6509 - val_acc: 0.0290

Epoch 00004: val_

In [11]:
params = {'network_name': "benchmark",
         'image_aug': False,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}

In [12]:
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 33s 58ms/step - loss: 4.4228 - acc: 0.0376 - val_loss: 4.0705 - val_acc: 0.0533

Epoch 00001: val_acc improved from -inf to 0.05331, saving model to /data/oxford102/experiments/1548039573.9481478/benchmark_1548039573.9481478.h5
Epoch 2/100
575/575 [==============================] - 32s 56ms/step - loss: 3.7344 - acc: 0.1211 - val_loss: 3.6032 - val_acc: 0.1372

Epoch 00002: val_acc improved from 0.05331 to 0.13720, saving model to /data/oxford102/experiments/1548039573.9481478/benchmark_1548039573.9481478.h5
Epoch 3/100
575/575 [==============================] - 32s 56ms/step - loss: 3.1667 - acc: 0.2126 - val_loss: 3.2118 - val_acc: 0.2127

Epoch 00003: val_acc improved from 0.13720 to 0.21271, saving model to /data/oxford102/experiments/1548039573.9481478/benchmark_1548039573.9481478.h5
Epoch 4/100
575/575 [==============================] - 32

In [8]:
def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(n_categories, activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [11]:
params = {'network_name': "benchmark",
         'image_aug': False,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 40s 70ms/step - loss: 4.3079 - acc: 0.0509 - val_loss: 3.8810 - val_acc: 0.0947

Epoch 00001: val_acc improved from -inf to 0.09467, saving model to /data/oxford102/experiments/1548040357.6217308/benchmark_1548040357.6217308.h5
Epoch 2/100
575/575 [==============================] - 39s 68ms/step - loss: 3.4832 - acc: 0.1561 - val_loss: 3.3817 - val_acc: 0.1796

Epoch 00002: val_acc improved from 0.09467 to 0.17956, saving model to /data/oxford102/experiments/1548040357.6217308/benchmark_1548040357.6217308.h5
Epoch 3/100
575/575 [==============================] - 39s 68ms/step - loss: 2.8557 - acc: 0.2787 - val_loss: 3.1480 - val_acc: 0.2357

Epoch 00003: val_acc improved from 0.17956 to 0.23573, saving model to /data/oxford102/experiments/1548040357.6217308/benchmark_1548040357.6217308.h5
Epoch 4/100
575/575 [==============================] - 39

In [9]:
def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.7))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.7))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.7))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))

    model.add(Dense(n_categories, activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [12]:
params = {'network_name': "benchmark",
         'image_aug': False,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 41s 71ms/step - loss: 4.3114 - acc: 0.0511 - val_loss: 3.9101 - val_acc: 0.0882

Epoch 00001: val_acc improved from -inf to 0.08824, saving model to /data/oxford102/experiments/1548040933.4639325/benchmark_1548040933.4639325.h5
Epoch 2/100
575/575 [==============================] - 39s 68ms/step - loss: 3.4808 - acc: 0.1598 - val_loss: 3.3776 - val_acc: 0.1869

Epoch 00002: val_acc improved from 0.08824 to 0.18692, saving model to /data/oxford102/experiments/1548040933.4639325/benchmark_1548040933.4639325.h5
Epoch 3/100
575/575 [==============================] - 39s 68ms/step - loss: 2.8374 - acc: 0.2809 - val_loss: 3.1235 - val_acc: 0.2431

Epoch 00003: val_acc improved from 0.18692 to 0.24309, saving model to /data/oxford102/experiments/1548040933.4639325/benchmark_1548040933.4639325.h5
Epoch 4/100
575/575 [==============================] - 39

In [ ]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

In [9]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.7))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.7))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.7))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))

    model.add(Dense(n_categories,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [12]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 41s 71ms/step - loss: 6.0723 - acc: 0.0487 - val_loss: 5.5043 - val_acc: 0.0818

Epoch 00001: val_acc improved from -inf to 0.08180, saving model to /data/oxford102/experiments/1548042086.4225383/benchmark_1548042086.4225383.h5
Epoch 2/100
575/575 [==============================] - 39s 69ms/step - loss: 4.9473 - acc: 0.1465 - val_loss: 4.7098 - val_acc: 0.1805

Epoch 00002: val_acc improved from 0.08180 to 0.18048, saving model to /data/oxford102/experiments/1548042086.4225383/benchmark_1548042086.4225383.h5
Epoch 3/100
575/575 [==============================] - 39s 68ms/step - loss: 4.1181 - acc: 0.2585 - val_loss: 4.3036 - val_acc: 0.2127

Epoch 00003: val_acc improved from 0.18048 to 0.21271, saving model to /data/oxford102/experiments/1548042086.4225383/benchmark_1548042086.4225383.h5
Epoch 4/100
575/575 [==============================] - 39

In [ ]:
keras.preprocessing.image.ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0, height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0, fill_mode='nearest', cval=0.0, horizontal_flip=False, vertical_flip=False, rescale=None, preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None)

In [9]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.85))

    model.add(Dense(n_categories,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [13]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 41s 72ms/step - loss: 6.0704 - acc: 0.0500 - val_loss: 5.5051 - val_acc: 0.0827

Epoch 00001: val_acc improved from -inf to 0.08272, saving model to /data/oxford102/experiments/1548043516.3301682/benchmark_1548043516.3301682.h5
Epoch 2/100
575/575 [==============================] - 39s 69ms/step - loss: 4.9436 - acc: 0.1533 - val_loss: 4.6930 - val_acc: 0.1768

Epoch 00002: val_acc improved from 0.08272 to 0.17680, saving model to /data/oxford102/experiments/1548043516.3301682/benchmark_1548043516.3301682.h5
Epoch 3/100
575/575 [==============================] - 39s 68ms/step - loss: 4.1052 - acc: 0.2620 - val_loss: 4.2645 - val_acc: 0.2265

Epoch 00003: val_acc improved from 0.17680 to 0.22652, saving model to /data/oxford102/experiments/1548043516.3301682/benchmark_1548043516.3301682.h5
Epoch 4/100
575/575 [==============================] - 39

In [9]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.85))

    model.add(Dense(n_categories,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [12]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 34s 59ms/step - loss: 6.2904 - acc: 0.0311 - val_loss: 6.0453 - val_acc: 0.0230

Epoch 00001: val_acc improved from -inf to 0.02298, saving model to /data/oxford102/experiments/1548044535.778602/benchmark_1548044535.778602.h5
Epoch 2/100
575/575 [==============================] - 32s 56ms/step - loss: 5.8792 - acc: 0.0291 - val_loss: 5.6865 - val_acc: 0.0525

Epoch 00002: val_acc improved from 0.02298 to 0.05249, saving model to /data/oxford102/experiments/1548044535.778602/benchmark_1548044535.778602.h5
Epoch 3/100
575/575 [==============================] - 32s 56ms/step - loss: 5.3775 - acc: 0.0543 - val_loss: 5.0505 - val_acc: 0.0654

Epoch 00003: val_acc improved from 0.05249 to 0.06538, saving model to /data/oxford102/experiments/1548044535.778602/benchmark_1548044535.778602.h5
Epoch 4/100
575/575 [==============================] - 32s 56ms

In [9]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer2
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
       
    # conv_layer3
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.85))

    model.add(Dense(n_categories,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [12]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 34s 58ms/step - loss: 6.2741 - acc: 0.0267 - val_loss: 5.9802 - val_acc: 0.0294

Epoch 00001: val_acc improved from -inf to 0.02941, saving model to /data/oxford102/experiments/1548045075.7719638/benchmark_1548045075.7719638.h5
Epoch 2/100
575/575 [==============================] - 33s 58ms/step - loss: 5.5656 - acc: 0.0613 - val_loss: 5.2883 - val_acc: 0.0820

Epoch 00002: val_acc improved from 0.02941 to 0.08195, saving model to /data/oxford102/experiments/1548045075.7719638/benchmark_1548045075.7719638.h5
Epoch 3/100
575/575 [==============================] - 33s 57ms/step - loss: 4.9901 - acc: 0.0939 - val_loss: 4.7323 - val_acc: 0.1087

Epoch 00003: val_acc improved from 0.08195 to 0.10866, saving model to /data/oxford102/experiments/1548045075.7719638/benchmark_1548045075.7719638.h5
Epoch 4/100
575/575 [==============================] - 33

In [9]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.85))

    model.add(Dense(n_categories,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [13]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 33s 58ms/step - loss: 4.7690 - acc: 0.0383 - val_loss: 4.4998 - val_acc: 0.0708

Epoch 00001: val_acc improved from -inf to 0.07077, saving model to /data/oxford102/experiments/1548046260.477109/benchmark_1548046260.477109.h5
Epoch 2/100
575/575 [==============================] - 32s 56ms/step - loss: 4.5399 - acc: 0.0802 - val_loss: 4.6874 - val_acc: 0.0773

Epoch 00002: val_acc improved from 0.07077 to 0.07735, saving model to /data/oxford102/experiments/1548046260.477109/benchmark_1548046260.477109.h5
Epoch 3/100
575/575 [==============================] - 32s 55ms/step - loss: 4.6268 - acc: 0.1072 - val_loss: 4.7501 - val_acc: 0.1298

Epoch 00003: val_acc improved from 0.07735 to 0.12983, saving model to /data/oxford102/experiments/1548046260.477109/benchmark_1548046260.477109.h5
Epoch 4/100
575/575 [==============================] - 32s 56ms

In [ ]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.8))
    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.85))

    model.add(Dense(n_categories,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [17]:
# LeNet-5


In [13]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

#     import keras 
#     from keras.models import Sequential
#     from keras.layers import Conv2D
#     from keras.layers import MaxPooling2D
#     from keras.layers import Flatten
#     from keras.layers import Dense
    
#     model = Sequential()
    #Layer 1
    #Conv Layer 1
    model.add(Conv2D(filters = 6, 
                     kernel_size = 5, 
                     strides = 1, 
                     activation = 'relu', 
                     input_shape = (width,height,3)))
    #Pooling layer 1
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    #Layer 2
    #Conv Layer 2
    model.add(Conv2D(filters = 16, 
                     kernel_size = 5,
                     strides = 1,
                     activation = 'relu',
                     input_shape = (14,14,6)))
    #Pooling Layer 2
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    #Flatten
    model.add(Flatten())
    #Layer 3
    #Fully connected layer 1
    model.add(Dense(units = 120, activation = 'relu'))
    #Layer 4
    #Fully connected layer 2
    model.add(Dense(units = 84, activation = 'relu'))
    #Layer 5
    #Output Layer
    model.add(Dense(units = 102, activation = 'softmax'))

    if verbose:
        model.summary()

    return model, None


In [16]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 43s 75ms/step - loss: 4.0879 - acc: 0.0843 - val_loss: 3.6113 - val_acc: 0.1333

Epoch 00001: val_acc improved from -inf to 0.13327, saving model to /data/oxford102/experiments/1548101569.2404804/benchmark_1548101569.2404804.h5
Epoch 2/100
575/575 [==============================] - 33s 58ms/step - loss: 3.0858 - acc: 0.2343 - val_loss: 3.1702 - val_acc: 0.2238

Epoch 00002: val_acc improved from 0.13327 to 0.22376, saving model to /data/oxford102/experiments/1548101569.2404804/benchmark_1548101569.2404804.h5
Epoch 3/100
575/575 [==============================] - 32s 56ms/step - loss: 1.9438 - acc: 0.4857 - val_loss: 3.5375 - val_acc: 0.2403

Epoch 00003: val_acc improved from 0.22376 to 0.24033, saving model to /data/oxford102/experiments/1548101569.2404804/benchmark_1548101569.2404804.h5
Epoch 4/100
575/575 [==============================] - 32

In [9]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

#     import keras 
#     from keras.models import Sequential
#     from keras.layers import Conv2D
#     from keras.layers import MaxPooling2D
#     from keras.layers import Flatten
#     from keras.layers import Dense
    
#     model = Sequential()
    #Layer 1
    #Conv Layer 1
    model.add(Conv2D(filters = 20, 
                     kernel_size = 5, 
                     strides = 1, 
                     activation = 'relu', 
                     input_shape = (width,height,3)))
    #Pooling layer 1
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    #Layer 2
    #Conv Layer 2
    model.add(Conv2D(filters = 50, 
                     kernel_size = 5,
                     strides = 1,
                     activation = 'relu'
                     ))
    #Pooling Layer 2
    model.add(MaxPooling2D(pool_size = 2, strides = 2))
    #Flatten
    model.add(Flatten())
    #Layer 3
    #Fully connected layer 1
    model.add(Dense(units = 500, activation = 'relu'))
#     #Layer 4
#     #Fully connected layer 2
#     model.add(Dense(units = 84, activation = 'relu'))
    model.add(Dropout(0.85))
    #Layer 5
    #Output Layer
    model.add(Dense(units = 102, activation = 'softmax'))

    if verbose:
        model.summary()

    return model, None


In [12]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 61s 107ms/step - loss: 4.1051 - acc: 0.0811 - val_loss: 3.6721 - val_acc: 0.1369

Epoch 00001: val_acc improved from -inf to 0.13695, saving model to /data/oxford102/experiments/1548104077.225572/benchmark_1548104077.225572.h5
Epoch 2/100
575/575 [==============================] - 60s 104ms/step - loss: 3.2583 - acc: 0.2067 - val_loss: 3.3027 - val_acc: 0.2053

Epoch 00002: val_acc improved from 0.13695 to 0.20534, saving model to /data/oxford102/experiments/1548104077.225572/benchmark_1548104077.225572.h5
Epoch 3/100
575/575 [==============================] - 60s 104ms/step - loss: 2.5864 - acc: 0.3330 - val_loss: 3.2194 - val_acc: 0.2578

Epoch 00003: val_acc improved from 0.20534 to 0.25783, saving model to /data/oxford102/experiments/1548104077.225572/benchmark_1548104077.225572.h5
Epoch 4/100
575/575 [==============================] - 60s 1

In [ ]:
optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [9]:
from keras import regularizers

def get_benchmark_model(verbose=False, img_size=(256, 256)):
    
    width, height = img_size
    model = None
    model = Sequential()

    #conv_layer1
    model.add(Conv2D(32, (3, 3), input_shape=(width, height, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # conv_layer2
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    # conv_layer3
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(1024))
    model.add(BatchNormalization())
    model.add(Activation('relu'))


    model.add(Dense(n_categories,kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01), activation="softmax"))

    if verbose:
        model.summary()

    return model, None

In [12]:
params = {'network_name': "benchmark",
         'image_aug': True,
         'optimizer_name': "SGD", 
         'optimizer': optimizers.SGD(),
         'optimizer_params': None, 
         'batch_size': 8,
         'epochs': 100,
         'image_size': (256, 256),
         'log_time': None}
lr=0.001
rho=0.9
epsilon=None
decay=0.0

_model, input_processor = get_model(params["network_name"], image_size=params["image_size"])
train_generator, validation_generator = get_generators( params["batch_size"], params["image_size"], None)
train_model(params, _model, (train_generator, validation_generator))

Found 4604 images belonging to 102 classes.
Found 1094 images belonging to 102 classes.
Epoch 1/100
575/575 [==============================] - 34s 60ms/step - loss: 6.3179 - acc: 0.0228 - val_loss: 5.9689 - val_acc: 0.0331

Epoch 00001: val_acc improved from -inf to 0.03309, saving model to /data/oxford102/experiments/1548105991.8424842/benchmark_1548105991.8424842.h5
Epoch 2/100
575/575 [==============================] - 33s 57ms/step - loss: 5.6653 - acc: 0.0463 - val_loss: 5.3951 - val_acc: 0.0552

Epoch 00002: val_acc improved from 0.03309 to 0.05525, saving model to /data/oxford102/experiments/1548105991.8424842/benchmark_1548105991.8424842.h5
Epoch 3/100
575/575 [==============================] - 33s 57ms/step - loss: 5.0083 - acc: 0.0902 - val_loss: 4.6999 - val_acc: 0.0976

Epoch 00003: val_acc improved from 0.05525 to 0.09761, saving model to /data/oxford102/experiments/1548105991.8424842/benchmark_1548105991.8424842.h5
Epoch 4/100
575/575 [==============================] - 33